In [4]:
from src.distances import dmat_and_order
import numpy as np
from scipy.sparse.csgraph import dijkstra
import networkx as nx
from ipywidgets import interact

In [5]:
from tqdm.notebook import tqdm

In [33]:
def spanner_adj(distances, t):
    ind = np.unravel_index(np.argsort(distances, axis=None), distances.shape)
    adj = np.zeros(distances.shape)
    total = 1.0*distances.shape[0] * distances.shape[1]
    for i,j in tqdm(zip(ind[0], ind[1]), total=len(ind[0])):
        if i<j: continue
        d = distances[i,j]
        shortest = dijkstra(adj, indices=i, min_only=True, limit=t*d)[j]
        if shortest <= t*d: continue
        adj[i,j] = d
        adj[j,i] = d
    return adj

In [41]:
adj = spanner_adj(dmat_sym, 5)

  0%|          | 0/279841 [00:00<?, ?it/s]

In [42]:
np.count_nonzero(adj)/np.count_nonzero(dmat_sym)

0.0038308414962479236

In [46]:
import networkx as nx

In [49]:
G = nx.from_numpy_matrix(dmat_sym)

In [109]:
G2 = nx.spanner(G, 2)
len(G2.edges)

139656

In [85]:
len(G.edges)

139656

In [96]:
type(G)

networkx.classes.graph.Graph